# HPE ILO Redfish - Prometheus - Grafana Monitoring

This Jupyter notebook describes the deployment of the monitoring of aHPE Server, using the ILO Redfish connector for Prometheus and the Grafana visualization tool 

The solution described is is deployed on a Kubernetes 1.20.10 cluster deployed in the Customer Technology Center Boeblingen, i.e. all parts of the monitoring stack are deployed as container.

## Connect to the Kubernetes environment

Check that you are connected to the correct Kubernetes cluster. If not use the ClusterConnection Notebook in the Toolkit to get connected to the correct Kubernetes cluster: suo04k8s. All Grafana Monitoring is located in the svtprometheus namespace. 

In [ ]:
kubectl get nodes
kubectl -n svtprometheus get pods,svc

## 1. Create the namespace

Create a namespace for your deployment. The example below creates the namespace svtprometheus. 

In [ ]:
# Example content of the namespace.yml file - either run the command above or this one to create the namespace
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
kind: Namespace
metadata:
  name: svtprometheus
EOF

## 2. Create the ConfigMap

The connection and monitoring parameters are transferred to the HPE ILO Redfish Prometheus connector as a Kubernetes ConfigMap. 

The ConfigMap can be created using th Python script: createILOcredentials.v1.0.py. 

Requirements:
    - System with Python 3 and the following Python packages installed: 
      - Fernet
      - getpass
      - etree

Run the script with the following command: python3 createILOcredentials.v1.0.py
The script will ask for the following information:
    - username               ILO username (requires read only access) 
    - password               ILO password
    - Logfile                Logfile name
    - Connector Port         TCP Port that the connector uses to publish the counters.
    - Filename               name of the yml-file (<name>.yml) and the configmap: <name>-xml
    - Monitoringintervall    Monitorintervall in seconds
    - ILO IP Address         ILO IP address (asks for it until you enter: 0)
    

It will create the Kubernetes yml-file (<name>.yml) that can be used to  create the Configmap <name>-xml 
    

In [ ]:
pwd
ls -l

In [ ]:
python createILOcredentials.v1.0.py

The configmap definition need to be applied now. 

In [ ]:
kubectl apply -f iloprometheus.yml

Check that the configmap is created properly

In [ ]:
kubectl -n svtprometheus get configmap

In [ ]:
kubectl -n svtprometheus describe configmap/iloprometheus-xml

## 3. Create the ILO Redfish Connector POD and Service with the Config Maps

After having the configmap created the ILO Redfish connector can be deployed now. The configmap will be mounted to /opt/prometheus/data and will provide the necessary configuration data for the connector. 

In [ ]:
cat << 'EOF' | kubectl apply -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: iloredfish
  namespace: svtprometheus
  labels:
    app: iloredfish
spec:
  selector:
    matchLabels:
      app: iloredfish
  template:
    metadata:
      labels:
        app: iloredfish
    spec:
      containers:
        - name: iloredfish
          image: tb1378/iloprometheus
          command: ["/usr/bin/python3"]
          args: ["/opt/prometheus/iloPromConnector.v1.0.py"]
          volumeMounts:
            - name: iloprometheusxml
              mountPath: /opt/prometheus/data
      volumes:
        - name: iloprometheusxml
          configMap:
            name: iloprometheus-xml   # the correct name of the configmap needs to be added here. 
EOF

The ILO Redfish connector need to be exported as a service in order to be able to collect the data with Prometheus.

In [ ]:
cat << 'EOF' | kubectl apply -f -
apiVersion: v1
kind: Service
metadata:
  name: iloredfish
  namespace: svtprometheus
#  labels: 
#     hpecp.hpe.com/hpecp-internal-gateway: "true"    
spec:
  selector:
    app: iloredfish
  ports:
    - port: 9091               # The Port of that the Prometheus connector uses
      targetPort: 9091
      protocol: TCP
  type: NodePort               # expose the Prometheus connector if you want/need to debug it. 
EOF

# Troubleshooting

First check the deployments, services and pods running in the namespaces.

In [ ]:
kubectl -n svtprometheus get deployments
kubectl -n svtprometheus get svc
kubectl -n svtprometheus get pods

Second check the logs of the pod.

In [ ]:
kubectl -n svtprometheus logs pod/iloredfish-5569d54664-rbc7b

# Delete the Deployment and the Service

In [ ]:
kubectl -n svtprometheus delete deployment/iloredfish

In [ ]:
kubect -n svtprometheus delete svc/iloredfish